# CSV to SQLite converter
We used this python script to convert the providers.csv to an sqlite database, when we upgrade from the csv to an actual api implementation for the data this can be removed. 

In [31]:
import pandas as pd
import sqlite3
import re

In [32]:
# Define the path to your CSV file
csv_file_path = './providers.csv'
sqlite_db_path = '../db/providers.db'

In [33]:
# Define constants for CSV reading
CSV_HEADER_OFFSET = 2  # skip first 2 rows
cursor = 0  # Adjust as needed
limit = None  # Adjust as needed

In [34]:
df = pd.read_csv(csv_file_path, skiprows=CSV_HEADER_OFFSET + cursor, nrows=limit, dtype=str)

In [35]:
def parse_int(value):
    try:
        return int(''.join(filter(str.isdigit, str(value)))) if pd.notnull(value) else None
    except ValueError:
        return None

def parse_float(value):
    try:
        cleaned_value = ''.join(filter(lambda x: x.isdigit() or x == '.', str(value)))
        # Handle cases with multiple periods (e.g., '3.53.9')
        parts = cleaned_value.split('.')
        if len(parts) > 2:
            cleaned_value = parts[0] + '.' + ''.join(parts[1:])
        return float(cleaned_value) if pd.notnull(value) else None
    except ValueError:
        return None

def parse_bool(value):
    return value.lower() == 'true' if pd.notnull(value) else None

# 
def parse_storage(storage_str):
    """
    Parse storage string and return the total storage in GB.
    """
    if pd.isnull(storage_str):
        return None
    
    # Handle the case with multiplications
    pattern = re.compile(r'(\d+)[xX]\s*(\d+\.?\d*)\s*(TB|GB)')
    match = pattern.search(storage_str)
    
    if match:
        quantity = int(match.group(1))
        size = float(match.group(2))
        unit = match.group(3)
        
        if unit.upper() == 'TB':
            size *= 1024  # Convert TB to GB
        
        return int(quantity * size)
    
    # Handle the case without multiplications
    pattern = re.compile(r'(\d+\.?\d*)\s*(TB|GB)')
    match = pattern.search(storage_str)
    
    if match:
        size = float(match.group(1))
        unit = match.group(2)
        
        if unit.upper() == 'TB':
            size *= 1024  # Convert TB to GB
        
        return int(size)
    
    # If no pattern matches, return None
    return None

def parse_ghz(value):
    if pd.isnull(value):
        return None
    
    value = str(value)
    
    if '/' in value:
        parts = value.split('/')
        if len(parts) > 1 and parts[1].strip():
            value = parts[1].strip()
        else:
            value = parts[0].strip()
    
    cleaned_value = ''.join(filter(lambda x: x.isdigit() or x == '.', value))
    
    try:
        return float(cleaned_value)
    except ValueError:
        return None
    

def parse_cores(value):
    if pd.isnull(value):
        return None
    
    value = str(value).strip()
    
    # Handle ranges like "4-8", "4 - 8", "12-32", "12 - 32 Cores"
    range_pattern = re.compile(r'(\d+)\s*-\s*(\d+)')
    match = range_pattern.search(value)
    
    if match:
        return int(match.group(2))
    
    # Handle single values like "8", "16"
    single_pattern = re.compile(r'(\d+)')
    match = single_pattern.match(value)
    
    if match:
        return int(match.group(1))
    
    # If no pattern matches, return None
    return None


In [36]:
def transform_row(row):
    if pd.isnull(row.iloc[1]):
        return None
    return {
        'providerName': row.iloc[1],
        'productName': row.iloc[2],
        'country': row.iloc[3],
        'location': row.iloc[4],
        'cpuCores': parse_cores(row.iloc[6]),
        'cpuThreads': parse_int(row.iloc[8]),
        'cpuGHZ': parse_ghz(row.iloc[9]),
        'hasSGX': parse_bool(row.iloc[10]),
        'ram': parse_int(row.iloc[41]),
        'numberDrives': parse_int(row.iloc[43]),
        'avgSizeDrive': parse_int(row.iloc[44]),
        'storageTotal': parse_storage(row.iloc[62]),
        'gpuType': row.iloc[63],
        'gpuMemory': f"{row.iloc[64]}_{row.iloc[65]}",
        'bandwidthNetwork': parse_int(row.iloc[67]),
        'network': parse_int(row.iloc[68]),
        'priceHour': parse_float(row.iloc[74]),
        'priceMonth': parse_float(row.iloc[77]),
        'priceSale': parse_float(row.iloc[78]),
        'availability': row.iloc[79],
        'source': row.iloc[80],
        'unit': row.iloc[82],
    }

In [37]:
# Apply the transformation to each row
transformed_data = df.apply(transform_row, axis=1).dropna().tolist()

In [38]:
# Convert the list of dictionaries to a DataFrame
transformed_df = pd.DataFrame(transformed_data)

In [39]:
# Ensure the directory for the database exists
import os
os.makedirs(os.path.dirname(sqlite_db_path), exist_ok=True)

In [40]:
# Write the transformed data to a SQLite database with an auto-incrementing id column
with sqlite3.connect(sqlite_db_path) as conn:
    cursor = conn.cursor()
    # Create table with an auto-incrementing id column
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS providers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            providerName TEXT,
            productName TEXT,
            country TEXT,
            location TEXT,
            cpuCores INTEGER,
            cpuThreads INTEGER,
            cpuGHZ REAL,
            hasSGX BOOLEAN,
            ram INTEGER,
            numberDrives INTEGER,
            avgSizeDrive INTEGER,
            storageTotal INTEGER,
            gpuType TEXT,
            gpuMemory TEXT,
            bandwidthNetwork INTEGER,
            network INTEGER,
            priceHour REAL,
            priceMonth REAL,
            availability TEXT,
            source TEXT,
            unit TEXT
        )
    ''')

    # Insert the DataFrame into the SQLite table
    for row in transformed_df.itertuples(index=False, name=None):
        cursor.execute('''
            INSERT INTO providers (
                providerName, productName, country, location, cpuCores, cpuThreads, cpuGHZ, hasSGX, ram, numberDrives, avgSizeDrive, storageTotal, gpuType, gpuMemory, bandwidthNetwork, network, priceHour, priceMonth, availability, source, unit
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', row)
    
    conn.commit()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 21, and there are 22 supplied.